In [ ]:
# %pip install -r requirements.txt

## Preprocessing
The preprocessing pipelines are defined in this notebook. Throughout the project, two pipelines are mainly used, the pipeline that cleans the data, so that it can be used to extract other features, and the pipeline that extracts the `signal` and `label` for a neural network.

In [ ]:
from sia import Preprocessing
from sia.io import Metadata, read_edf, read_csv, write_csv
from sia.preprocessors import neurokit

In [ ]:
from sia.encoders import GroupEncoder
from sklearn.preprocessing import LabelEncoder

### Cleaning Pipeline
The pipeline defined below uses the default preprocessor of Neurokit to clean the data. Furthermore, the data with a low quality is also discarded.

In [ ]:
Preprocessing() \
    .data(
        read_edf(
            './data/raw/*.edf', 
            Metadata('./data/raw/TimeStamps_Merged.txt').on_regex(r'[0-9]{5}')
        )
    ) \
    .rename({'category': 'label'}) \
    .encode({'label': 'category'}, GroupEncoder({
        'baseline': ['Sitting', 'Recov1', 'Recov2', 'Recov3', 'Recov4', 'Recov5', 'Recov6'],
        'mental_stress': ['TA', 'SSST_Sing_countdown', 'Pasat', 'Raven', 'TA_repeat', 'Pasat_repeat'],
        'high_physical_activity': ['Treadmill1', 'Treadmill2', 'Treadmill3', 'Treadmill4', 'Walking_fast_pace', 'Cycling', 'stairs_up_and_down'],
        'moderate_physical_activity': ['Walking_own_pace', 'Dishes', 'Vacuum'],
        'low_physical_activity': ['Standing', 'Lying_supine', 'Recov_standing']
    })) \
    .filter(lambda category: [_category != None for _category in category]) \
    .process(neurokit()) \
    .filter(lambda ECG_Quality: [quality > .25 for quality in ECG_Quality]) \
    .to(write_csv('./data/cleaned/[0-9]{5}.csv'))

#### HuggingFace Loader
The project often uses HuggingFace Dataset to load the data. To load datadirectories, [HuggingFace Dataset requires a dataloader file](https://huggingface.co/docs/datasets/en/loading) to know how the data is structured.
 
SiA-Kit defines a default "multi-participant" dataloader in `sia.config` that can be used to load the data. This can be used by creating a file in the written directory with the same name as the directory and the extension `.py`. 

An example can be seen below,
```python
## Filename, `cleaned.py`
from datasets.packaged_modules import csv
from datasets import Features, Value
from sia.config import MultiParticipant, MultiParticipantConfig

class CleanedConfig(MultiParticipantConfig, csv.CsvConfig):
    def __init__(
        self,
        train_participants: list[Union[str, int]] = [],
        val_participants: list[Union[str, int]] = [],
        test_participants: list[Union[str, int]] = [],
        *args,
        **kwargs
    ):
        """
        Parameters
        ----------
        train_participants : list[Union[str, int]]
            List of participants to be used for training.
        val_participants : list[Union[str, int]]
            List of participants to be used for validation. 
        test_participants : list[Union[str, int]]
            List of participants to be used for testing.
        """
        MultiParticipantConfig.__init__(self, train_participants, val_participants, test_participants)
        csv.CsvConfig.__init__(self, *args, **kwargs)

class Cleaned(MultiParticipant, csv.Csv):
    BUILDER_CONFIG_CLASS = CleanedConfig

    BUILDER_CONFIGS = [
        CleanedConfig(
            name="cleaned", 
            description="Cleaned data",
            version="1.0.0", 
            skiprows=1,
            column_names=[
                'category',
                ...
            ],
            features=Features({
                'category': Value('string'),
                ...
            })
        )
    ]

    DEFAULT_CONFIG_NAME = "cleaned"

### Neural Network 
For the neural network, only the signal and the label is extracted and saved in another directory.

In [ ]:
Preprocessing() \
    .data(read_csv('./data/cleaned/*.csv', columns=['ECG_Clean', 'category'])) \
    .rename({'ECG_Clean': 'signal'}) \
    .encode('category', LabelEncoder()) \
    .to(write_csv('./data/signal/[0-9]{5}.csv'))